# Groupwise cortical parcellation

In [ ]:
# ||AUM||
%matplotlib inline
%gui qt
import scipy.io
import scipy as sp
import numpy as np
from dfsio import readdfs, writedfs
from mayavi import mlab
#import h5py
import os
from surfproc import view_patch, view_patch_vtk, get_cmap, smooth_patch
from sklearn.cluster import SpectralClustering
from sklearn.decomposition import DictionaryLearning
from scipy.stats import trim_mean
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.utils.linear_assignment_ import linear_assignment
from sklearn.metrics import silhouette_score
p_dir = 'E:\\HCP-fMRI-NLM'
p_dir_ref='E:\\'
lst = os.listdir(p_dir)
r_factor = 3
ref_dir = os.path.join(p_dir_ref, 'reference')
nClusters=2

ref = '100307'
sub = lst[15]
print sub, ref
print(ref + '.reduce' + str(r_factor) + '.LR_mask.mat')
fn1 = ref + '.reduce' + str(r_factor) + '.LR_mask.mat'
fname1 = os.path.join(ref_dir, fn1)
msk = scipy.io.loadmat(fname1)  # h5py.File(fname1);
dfs_left = readdfs(os.path.join(p_dir_ref, 'reference', ref + '.aparc.a2009s.32k_fs.reduce3.left.dfs'))
dfs_left_sm = readdfs(os.path.join(p_dir_ref, 'reference', ref + '.aparc.a2009s.32k_fs.reduce3.very_smooth.left.dfs'))
count1 = 0
roilist=[30, 72, 9, 47] #pc
ref=lst[11]
datasub = scipy.io.loadmat(os.path.join(p_dir, sub, sub + '.rfMRI_REST1_RL.reduce3.ftdata.NLM_11N_hvar_25.mat'))
dataref = scipy.io.loadmat(os.path.join(p_dir, ref, ref + '.rfMRI_REST1_RL.reduce3.ftdata.NLM_11N_hvar_25.mat'))

LR_flag = msk['LR_flag']
LR_flag = np.squeeze(LR_flag) > 0
data = dataref['ftdata_NLM']
temp = data[LR_flag, :]
m = np.mean(temp, 1)
temp = temp - m[:,None]
s = np.std(temp, 1)+1e-16
temp = temp/s[:,None]

data = datasub['ftdata_NLM']
tempsub = data[LR_flag, :]
m = np.mean(tempsub, 1)
tempsub = tempsub - m[:,None]
s = np.std(tempsub, 1)+1e-16
tempsub = tempsub/s[:,None]

msk_small_region = np.in1d(dfs_left.labels,roilist)
#    msk_small_region = (dfs_left.labels == 30) | (dfs_left.labels == 72) | (dfs_left.labels == 9) |  (dfs_left.labels == 47)  # % motor
d = temp[msk_small_region, :]

ref_mean_pc = sp.mean(d,axis=0)
ref_mean_pc=ref_mean_pc-sp.mean(ref_mean_pc)
ref_mean_pc=ref_mean_pc/(sp.std(ref_mean_pc))

rho = np.dot(ref_mean_pc,temp.T)
rho[~np.isfinite(rho)] = 0
dfs_left_sm.attributes = rho
view_patch(dfs_left_sm,rho)

rho = np.dot(ref_mean_pc,tempsub.T)
rho[~np.isfinite(rho)] = 0
dfs_left.attributes = rho
view_patch(dfs_left_sm,rho)

sm=smooth_patch(dfs_left,iter=1000)
view_patch_vtk(sm)
#view_patch(dfs_left,rho)

xcorr=sp.dot((tempsub.T),temp)

u,s,v=scipy.linalg.svd(xcorr)
R=sp.dot(v.T,u.T)

rho = sp.dot(ref_mean_pc,sp.dot(tempsub,R.T).T)
#rho=rho[0,1:]
rho[~np.isfinite(rho)] = 0
dfs_left.attributes = rho
view_patch(dfs_left_sm,rho)

130013 100307
100307.reduce3.LR_mask.mat
rendering!


In [2]:
import os
from dfsio import readdfs,writedfs
import networkx as nx
g = nx.Graph()
p_dir = 'E:\\HCP-fMRI-NLM';
sub = '100307';
s = readdfs(os.path.join(p_dir, 'reference', sub + '.aparc.a2009s.32k_fs.reduce3.very_smooth.left.dfs'));
g.add_edges_from(s.faces[:,(0,1)]);g.add_edges_from(s.faces[:,(1,2)]);g.add_edges_from(s.faces[:,(2,0)]);
Adj=nx.adjacency_matrix(g)

IOError: File name E:\HCP-fMRI-NLM\reference\100307.aparc.a2009s.32k_fs.reduce3.very_smooth.left.dfs does not exist.

In [3]:
AdjS=Adj[(msk_small_region),:];AdjS=AdjS[:,(msk_small_region)]
AdjS=AdjS.todense()
#AdjS=np.matrix(AdjS)
np.fill_diagonal(AdjS,1)
#AdjS.shape
AdjS=1.0*((AdjS*AdjS*AdjS)>0)
rho_rho=np.multiply(AdjS,rho_rho)
np.sum(AdjS,1)
type(AdjS)

NameError: name 'Adj' is not defined

In [4]:
from sklearn.covariance import GraphLassoCV
dd=np.concatenate((d,d_corr),0)
model = GraphLassoCV(mode='lars')
model.fit(dd) 
rho2 = model.covariance_
rho2=rho2[range(d.shape[0]),d.shape[0]+1:]
rho2[~np.isfinite(rho2)] = 0;
model.fit(rho2)
rho_rho2=model.covariance_

NameError: name 'd' is not defined

In [1]:
rho = model.covariance_
rho=rho[range(d.shape[0]),d.shape[0]+1:]
rho[~np.isfinite(rho)] = 0;
model.fit(rho)
rho_rho=model.covariance_

NameError: name 'model' is not defined

In [11]:
np.fill_diagonal(rho_rho,0)

In [18]:
#from skimage import data, io, segmentation, color
#from skimage.future import graph
from sklearn.cluster import SpectralClustering

In [13]:
AdjS

matrix([[ 1.,  1.,  1., ...,  0.,  0.,  0.],
        [ 1.,  1.,  1., ...,  0.,  0.,  0.],
        [ 1.,  1.,  1., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  1.,  1.,  1.],
        [ 0.,  0.,  0., ...,  1.,  1.,  1.],
        [ 0.,  0.,  0., ...,  1.,  1.,  1.]])

In [21]:
for nclusters in [2, 5, 10, 30,200]:
    SC=SpectralClustering(n_clusters=nclusters,affinity='precomputed')
    labs=SC.fit_predict(rho_rho)

    r=dfs_left_sm;r.labels=r.labels*0;r.labels[msk_small_region]=labs+1;
    mesh = mlab.triangular_mesh(r.vertices[:,0], r.vertices[:,1], r.vertices[:,2], r.faces, representation='surface', 
                            opacity=1,scalars=np.float64(r.labels))
    #mlab.pipeline.surface(mesh)

    mlab.gcf().scene.parallel_projection = True
    mlab.view(azimuth=0, elevation=-90)
    mlab.savefig(filename = sub+str(nclusters)+'labels1.png')
    mlab.view(azimuth=0, elevation=90)
    mlab.savefig(filename = sub+str(nclusters)+'labels2.png')
    mlab.close();

In [6]:
str(5)

'5'

In [7]:
np.min(labs)

0